# Excel 分割器 - OneDrive 簡化版

<!-- 
開發日期: 2025年1月
功能: 專為 OneDrive 使用者設計的簡化版本
特色: 本地處理，保留資料驗證
-->

這個筆記本專為使用 OneDrive 的使用者設計，可在本地處理 Excel 檔案並保留資料驗證規則。

## 主要特色
- 📂 自動建立審查者專屬資料夾
- 📊 按審查者分割 Excel 檔案
- ✅ **保留資料驗證規則**
- 📁 本地處理，無需連線設定
- 🚀 簡化的使用流程

## 系統需求
- Python 3.9 或更高版本
- Microsoft Office（建議）
- OneDrive 同步資料夾

## 步驟 1: 安裝必要套件

In [ ]:
# 安裝必要套件
import sys
import subprocess
import platform

required_packages = [
    'pandas',
    'openpyxl',
    'ipywidgets',
    'xlsxwriter'
]

# Windows 特定套件
if platform.system() == 'Windows':
    required_packages.append('pywin32')

print("🔍 檢查套件安裝狀態...")
for package in required_packages:
    try:
        if package == 'pywin32':
            __import__('win32com.client')
        else:
            __import__(package.replace('-', '_'))
        print(f"✓ {package} 已安裝")
    except ImportError:
        print(f"📦 正在安裝 {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} 安裝完成")

print("\n✅ 所有必要套件已就緒！")

## 步驟 2: 匯入函式庫與設定

In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import glob
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import time
from typing import Dict, List, Optional, Tuple
import re

# 檢查 tkinter (檔案對話框)
try:
    import tkinter as tk
    from tkinter import filedialog, messagebox
    TKINTER_AVAILABLE = True
except ImportError:
    TKINTER_AVAILABLE = False

# 檢查 win32com (Windows Excel 自動化)
WIN32COM_AVAILABLE = False
if platform.system() == 'Windows':
    try:
        import win32com.client
        WIN32COM_AVAILABLE = True
    except ImportError:
        pass

print("✓ 函式庫匯入成功")
print(f"✓ 作業系統: {platform.system()}")
print(f"✓ 檔案對話框: {'可用' if TKINTER_AVAILABLE else '不可用'}")
print(f"✓ Excel 自動化: {'可用' if WIN32COM_AVAILABLE else '不可用'}")

# 全域變數
last_folder = os.path.expanduser("~")

## 步驟 3: Excel 處理功能（保留資料驗證）

In [ ]:
def sanitize_folder_name(name: str) -> str:
    """清理資料夾名稱，確保相容性"""
    # 移除或替換不允許的字元
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '#', '%']
    sanitized = name.strip()
    
    for char in invalid_chars:
        sanitized = sanitized.replace(char, '_')
    
    # 限制長度
    if len(sanitized) > 255:
        sanitized = sanitized[:255].rstrip()
    
    return sanitized

def find_column(worksheet, column_name):
    """在工作表中尋找欄位"""
    for col_idx, cell in enumerate(worksheet[1], start=1):
        if cell.value == column_name:
            return col_idx
    raise ValueError(f"找不到 '{column_name}' 欄位！")

def copy_selected_documents(source_dir, dest_dir, copy_word=True, copy_pdf=True):
    """複製選定的文件類型"""
    copied_files = []
    
    if copy_word:
        word_patterns = [
            os.path.join(source_dir, "*.docx"),
            os.path.join(source_dir, "*.doc")
        ]
        
        for pattern in word_patterns:
            for file in glob.glob(pattern):
                if os.path.isfile(file):
                    dest_path = os.path.join(dest_dir, os.path.basename(file))
                    shutil.copy2(file, dest_path)
                    copied_files.append(os.path.basename(file))
    
    if copy_pdf:
        pdf_pattern = os.path.join(source_dir, "*.pdf")
        for file in glob.glob(pdf_pattern):
            if os.path.isfile(file):
                dest_path = os.path.join(dest_dir, os.path.basename(file))
                shutil.copy2(file, dest_path)
                copied_files.append(os.path.basename(file))
    
    return copied_files

def process_reviewer_excel_copy_first(file_path, reviewer, column_name, output_folder):
    """使用方案 2: 先複製整個檔案再處理（保留資料驗證）"""
    try:
        # 清理審查者名稱
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        
        # 建立審查者資料夾
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        # 建立新檔名
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        ext = os.path.splitext(base_name)[1]
        new_filename = f"{name_without_ext} - {reviewer_name}{ext}"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 先複製整個檔案
        shutil.copy2(file_path, dst_path)
        print(f"  ✓ 已複製檔案: {new_filename}")
        
        # 使用 openpyxl 處理複製的檔案
        wb = load_workbook(dst_path, data_only=False, keep_vba=True, keep_links=True)
        main_ws = wb.active
        
        # 保留所有資料驗證資訊
        data_validations = list(main_ws.data_validations.dataValidation)
        
        # 尋找並刪除非相關列
        col_idx = find_column(main_ws, column_name)
        rows_to_delete = []
        
        for row in range(2, main_ws.max_row + 1):
            cell_value = main_ws.cell(row=row, column=col_idx).value
            if str(cell_value) != str(reviewer):
                rows_to_delete.append(row)
        
        print(f"  ✓ 找到 {len(rows_to_delete)} 列需要刪除")
        
        # 從底部開始刪除
        for row in sorted(rows_to_delete, reverse=True):
            main_ws.delete_rows(row)
        
        # 重新應用資料驗證
        for dv in data_validations:
            main_ws.add_data_validation(dv)
        
        # 儲存變更
        wb.save(dst_path)
        wb.close()
        
        print(f"  ✓ 已處理完成，保留資料驗證")
        
        return True, reviewer_folder, new_filename
        
    except Exception as e:
        print(f"❌ 處理 {reviewer} 的檔案時發生錯誤: {str(e)}")
        return False, None, None

def process_reviewer_excel_windows_excel(file_path, reviewer, column_name, output_folder):
    """使用 Windows Excel COM 自動化（最可靠）"""
    if not WIN32COM_AVAILABLE:
        return False, None, None
        
    try:
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        new_filename = f"{name_without_ext} - {reviewer_name}.xlsx"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 啟動 Excel
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False
        excel.DisplayAlerts = False
        
        try:
            # 開啟原始檔案
            wb = excel.Workbooks.Open(os.path.abspath(file_path))
            ws = wb.ActiveSheet
            
            # 尋找欄位
            col_idx = None
            for col in range(1, ws.UsedRange.Columns.Count + 1):
                if ws.Cells(1, col).Value == column_name:
                    col_idx = col
                    break
            
            if not col_idx:
                raise ValueError(f"找不到欄位 '{column_name}'")
            
            # 套用篩選
            ws.AutoFilterMode = False
            ws.UsedRange.AutoFilter(Field=col_idx, Criteria1=str(reviewer))
            
            # 刪除隱藏的列
            for row in range(ws.UsedRange.Rows.Count, 1, -1):
                if ws.Rows(row).Hidden:
                    ws.Rows(row).Delete()
            
            # 移除篩選
            ws.AutoFilterMode = False
            
            # 另存新檔
            wb.SaveAs(os.path.abspath(dst_path))
            wb.Close()
            
            print(f"  ✓ 使用 Excel COM 處理完成")
            return True, reviewer_folder, new_filename
            
        finally:
            excel.Quit()
            
    except Exception as e:
        print(f"❌ Excel COM 處理失敗: {str(e)}")
        return False, None, None

print("✓ Excel 處理功能已就緒")

## 步驟 4: 使用者介面

In [ ]:
# === 檔案選擇 ===
display(HTML("<h3>📁 步驟 1: 檔案選擇</h3>"))

def select_excel_file():
    """選擇 Excel 檔案"""
    global last_folder
    if not TKINTER_AVAILABLE:
        print("❌ 檔案對話框不可用，請手動輸入檔案路徑")
        return
    
    try:
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)
        
        file_path = filedialog.askopenfilename(
            title="選擇 Excel 檔案",
            filetypes=[(
                "Excel 檔案", "*.xlsx *.xls"),
                ("所有檔案", "*.*")
            ],
            initialdir=last_folder
        )
        
        root.destroy()
        
        if file_path:
            excel_file_input.value = file_path
            last_folder = os.path.dirname(file_path)
            
            # 自動設定輸出資料夾為同一目錄
            output_folder_input.value = os.path.dirname(file_path)
            
            with output:
                print(f"✓ 已選擇: {os.path.basename(file_path)}")
                print(f"✓ 輸出資料夾: {os.path.dirname(file_path)}")
            
    except Exception as e:
        print(f"❌ 選擇檔案時發生錯誤: {str(e)}")

excel_file_input = widgets.Text(
    value='',
    placeholder='選擇 Excel 檔案...',
    description='Excel 檔案:',
    layout=widgets.Layout(width='450px')
)

browse_button = widgets.Button(
    description='瀏覽...',
    layout=widgets.Layout(width='100px')
)

browse_button.on_click(lambda x: select_excel_file())

reviewer_column_input = widgets.Text(
    value='Reviewer',
    description='審查者欄位:',
    layout=widgets.Layout(width='300px')
)

output_folder_input = widgets.Text(
    value='',
    placeholder='輸出資料夾（預設為 Excel 檔案所在資料夾）',
    description='輸出資料夾:',
    layout=widgets.Layout(width='450px')
)

display(widgets.HBox([excel_file_input, browse_button]))
display(reviewer_column_input)
display(output_folder_input)

# === 選項設定 ===
display(HTML("<h3>⚙️ 步驟 2: 選項設定</h3>"))

copy_word_check = widgets.Checkbox(
    value=True,
    description='複製 Word 文件 (.doc, .docx)'
)

copy_pdf_check = widgets.Checkbox(
    value=True,
    description='複製 PDF 文件 (.pdf)'
)

# 處理方法選擇
processing_method = widgets.RadioButtons(
    options=[
        ('複製檔案後處理（推薦）', 'copy_file_first'),
        ('使用 Excel COM 自動化（僅 Windows）', 'excel_com')
    ],
    value='copy_file_first',
    description='處理方法:',
    disabled=False
)

# 如果不是 Windows，隱藏 Excel COM 選項
if not WIN32COM_AVAILABLE:
    processing_method = widgets.RadioButtons(
        options=[
            ('複製檔案後處理', 'copy_file_first')
        ],
        value='copy_file_first',
        description='處理方法:',
        disabled=False
    )

display(copy_word_check)
display(copy_pdf_check)
display(HTML("<br><b>Excel 處理方法:</b>"))
display(processing_method)

# === 處理按鈕 ===
process_button = widgets.Button(
    description='處理 Excel 檔案',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

# === 輸出區域 ===
output = widgets.Output()

display(HTML("<br>"))
display(process_button)
display(output)

print("\n✅ 介面已就緒！")

## 步驟 5: 主要處理函數

In [ ]:
def process_excel_file(button):
    """主要處理函數"""
    with output:
        clear_output()
        
        # 驗證輸入
        if not excel_file_input.value:
            print("❌ 請選擇 Excel 檔案")
            return
        
        file_path = excel_file_input.value.strip()
        column_name = reviewer_column_input.value.strip()
        output_folder = output_folder_input.value.strip() or os.path.dirname(file_path)
        method = processing_method.value
        
        if not os.path.exists(file_path):
            print(f"❌ 找不到檔案: {file_path}")
            return
        
        print(f"📁 處理中: {os.path.basename(file_path)}")
        print(f"📊 審查者欄位: {column_name}")
        print(f"📂 輸出資料夾: {output_folder}")
        print(f"🔧 處理方法: {method}")
        print("=" * 50)
        
        # 開始計時
        start_time = time.time()
        
        try:
            # 讀取 Excel 檔案
            df = pd.read_excel(file_path, engine='openpyxl')
            
            if column_name not in df.columns:
                print(f"❌ 找不到欄位 '{column_name}'")
                print(f"可用欄位: {', '.join(df.columns)}")
                return
            
            # 取得唯一審查者
            reviewers = df[column_name].dropna().unique().tolist()
            print(f"✓ 找到 {len(reviewers)} 位審查者")
            
            # 建立進度條
            progress = widgets.IntProgress(
                value=0,
                min=0,
                max=len(reviewers),
                description='進度:',
                bar_style='info'
            )
            display(progress)
            
            # 處理每位審查者
            processed = 0
            failed = 0
            
            for i, reviewer in enumerate(reviewers):
                print(f"\n📝 處理中: {reviewer} ({i+1}/{len(reviewers)})")
                
                # 根據選擇的方法處理
                if method == 'excel_com' and WIN32COM_AVAILABLE:
                    success, folder_path, filename = process_reviewer_excel_windows_excel(
                        file_path, reviewer, column_name, output_folder
                    )
                else:
                    success, folder_path, filename = process_reviewer_excel_copy_first(
                        file_path, reviewer, column_name, output_folder
                    )
                
                if success:
                    # 複製相關文件
                    if copy_word_check.value or copy_pdf_check.value:
                        copied = copy_selected_documents(
                            os.path.dirname(file_path), folder_path,
                            copy_word=copy_word_check.value,
                            copy_pdf=copy_pdf_check.value
                        )
                        if copied:
                            print(f"  ✓ 已複製 {len(copied)} 個文件")
                    
                    processed += 1
                else:
                    failed += 1
                
                # 更新進度條
                progress.value = i + 1
            
            # 計算處理時間
            elapsed_time = time.time() - start_time
            
            # 總結
            print("\n" + "=" * 50)
            print(f"✅ 處理完成！")
            print(f"📊 成功處理: {processed}/{len(reviewers)} 位審查者")
            if failed > 0:
                print(f"❌ 處理失敗: {failed} 位")
            print(f"⏱️ 處理時間: {elapsed_time:.1f} 秒")
            print(f"📁 輸出位置: {output_folder}")
            
            # 顯示完成訊息
            print("\n💡 後續步驟：")
            print("1. 檢查輸出資料夾中的檔案")
            print("2. 確認資料驗證規則是否正常")
            print("3. 將資料夾上傳到 OneDrive 或 SharePoint")
            print("4. 分享給相應的審查者")
            
            # 提供開啟資料夾的選項
            if platform.system() == 'Windows':
                print(f"\n📂 <a href='file:///{output_folder.replace(chr(92), '/')}' target='_blank'>開啟輸出資料夾</a>")
            elif platform.system() == 'Darwin':  # macOS
                print(f"\n📂 在 Finder 中開啟: {output_folder}")
            
        except Exception as e:
            print(f"\n❌ 發生錯誤: {str(e)}")
            import traceback
            traceback.print_exc()

# 連接處理函數到按鈕
process_button.on_click(process_excel_file)
print("✓ 主要處理函數已就緒")

## 使用說明

### 快速開始

1. **選擇 Excel 檔案**
   - 點擊『瀏覽...』選擇要處理的 Excel 檔案
   - 輸出資料夾會自動設定為檔案所在位置

2. **確認設定**
   - 審查者欄位名稱（預設為 'Reviewer'）
   - 選擇要複製的文件類型
   - 選擇處理方法

3. **處理檔案**
   - 點擊『處理 Excel 檔案』開始處理
   - 查看進度條和處理狀態

4. **完成後續步驟**
   - 檢查輸出資料夾中的檔案
   - 將資料夾拖曳到 OneDrive 同步資料夾
   - 在 OneDrive 網頁版分享給審查者

### 處理方法說明

1. **複製檔案後處理（推薦）**
   - 先複製整個 Excel 檔案
   - 再刪除不相關的資料列
   - 保留所有資料驗證規則
   - 跨平台相容

2. **使用 Excel COM 自動化（僅 Windows）**
   - 使用 Microsoft Excel 程式處理
   - 最可靠但速度較慢
   - 需要安裝 Microsoft Excel

### 功能特色

- ✅ **保留資料驗證**：完整保留 Excel 資料驗證規則
- ✅ **批次處理**：一次處理多位審查者
- ✅ **自動複製文件**：同時複製 Word 和 PDF 文件
- ✅ **進度追蹤**：即時顯示處理進度
- ✅ **本地處理**：無需網路連線

### 疑難排解

1. **找不到欄位**
   - 確認欄位名稱正確（區分大小寫）
   - 檢查 Excel 檔案第一列是否為標題列

2. **資料驗證遺失**
   - 使用「複製檔案後處理」方法
   - 確認原始檔案的資料驗證設定正確
   - Windows 使用者可嘗試 Excel COM 方法

3. **處理速度慢**
   - 大型檔案處理需要時間
   - 可考慮先關閉其他程式
   - 確保有足夠的硬碟空間

### OneDrive 分享提示

1. 將產生的資料夾拖曳到 OneDrive 同步資料夾
2. 等待同步完成（查看 OneDrive 圖示）
3. 在 OneDrive 網頁版中：
   - 右鍵點擊資料夾
   - 選擇「分享」
   - 輸入審查者的 email
   - 設定適當的權限（檢視或編輯）
4. 或使用 OneDrive 桌面版的分享功能